In [1]:
import math
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image, UnidentifiedImageError
from pathlib import Path
import torch
import glob
import pytorch_lightning as pl
from huggingface_hub import HfApi, Repository
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
from torchmetrics import Accuracy
from transformers import ViTFeatureExtractor, ViTForImageClassification
from pytorch_lightning.callbacks import ModelCheckpoint
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay

In [2]:
data_dir = Path("/home/orelit/Projects -Sathsara/Planigo/data/Canned_food_Splitted/folder6")

ds=ImageFolder(data_dir)
indices = torch.randperm(len(ds)).tolist()
n_val = math.floor(len(indices) * .15)
train_ds = torch.utils.data.Subset(ds, indices[:-n_val])
val_ds = torch.utils.data.Subset(ds, indices[-n_val:])


label2id = {}
id2label = {}
for i, class_name in enumerate(ds.classes):
  label2id[class_name] = str(i)
  id2label[str(i)] = class_name

FileNotFoundError: [Errno 2] No such file or directory: '/home/orelit/Projects -Sathsara/Planigo/data/Canned_food_Splitted/folder6'

In [19]:
label2id = {}
id2label = {}
for i, class_name in enumerate(ds.classes):
  label2id[class_name] = str(i)
  id2label[str(i)] = class_name

In [20]:
import json
with open('/home/orelit/Projects -Sathsara/Planigo/Models/VIT_CF/model7/label2id.json', 'w') as fp:
    json.dump(label2id, fp)
    
with open('/home/orelit/Projects -Sathsara/Planigo/Models/VIT_CF/model7/id2label.json', 'w') as fp:
    json.dump(id2label, fp)

In [21]:
class ImageClassificationCollator:
   def __init__(self, feature_extractor): 
      self.feature_extractor = feature_extractor
   def __call__(self, batch):  
      encodings = self.feature_extractor([x[0] for x in batch],
      return_tensors='pt')   
      encodings['labels'] = torch.tensor([x[1] for x in batch],    
      dtype=torch.long)
      return encodings

feature_extractor = ViTFeatureExtractor.from_pretrained('google/vit-base-patch16-224-in21k')
collator = ImageClassificationCollator(feature_extractor)
train_loader = DataLoader(train_ds, batch_size=32, 
   collate_fn=collator, num_workers=2, shuffle=True)
val_loader = DataLoader(val_ds, batch_size=32, collate_fn=collator, 
   num_workers=2)
model = ViTForImageClassification.from_pretrained(
        'google/vit-base-patch16-224-in21k',
         num_labels=len(label2id),
         label2id=label2id,
         id2label=id2label)

Some weights of the model checkpoint at google/vit-base-patch16-224-in21k were not used when initializing ViTForImageClassification: ['pooler.dense.weight', 'pooler.dense.bias']
- This IS expected if you are initializing ViTForImageClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ViTForImageClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ViTForImageClassification were not initialized from the model checkpoint at google/vit-base-patch16-224-in21k and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [22]:
class Classifier(pl.LightningModule):
   def __init__(self, model, lr: float = 2e-5, **kwargs): 
       super().__init__()
       self.save_hyperparameters('lr', *list(kwargs))
       self.model = model
       self.forward = self.model.forward 
       self.val_acc = Accuracy()
   def training_step(self, batch, batch_idx):
       outputs = self(**batch)
       self.log(f"train_loss", outputs.loss)
       return outputs.loss
   def validation_step(self, batch, batch_idx):
       outputs = self(**batch)
       self.log(f"val_loss", outputs.loss)
       acc = self.val_acc(outputs.logits.argmax(1), batch['labels'])
       self.log(f"val_acc", acc, prog_bar=True)
       return outputs.loss
   def configure_optimizers(self):
       return torch.optim.Adam(self.parameters(), 
                        lr=self.hparams.lr,weight_decay = 0.00025)

In [23]:
pl.seed_everything(42)
classifier = Classifier(model, lr=2e-5)
trainer = pl.Trainer(gpus=1, precision=16, max_epochs=10)
trainer.fit(classifier, train_loader, val_loader)
model.save_pretrained("/home/orelit/Projects -Sathsara/Planigo/Models/VIT_CF/model7/model7")

Global seed set to 42
Using 16bit native Automatic Mixed Precision (AMP)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name    | Type                      | Params
------------------------------------------------------
0 | model   | ViTForImageClassification | 85.8 M
1 | val_acc | Accuracy                  | 0     
------------------------------------------------------
85.8 M    Trainable params
0         Non-trainable params
85.8 M    Total params
171.673   Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

`Trainer.fit` stopped: `max_epochs=10` reached.


In [8]:
# model.save_pretrained("/home/orelit/Projects -Sathsara/Planigo/Models/VIT_CF/model3/model3")

In [2]:
import pandas as pd
import json

model_names = ['model1','model2','model3','model4','model5','model6','model7','model8',
               'model9']

data_dict = {}
for model_name in model_names:
    with open("/home/orelit/Projects -Sathsara/Planigo/Models/VIT_CF/id2label_{}.json"
                   .format(model_name), 'r') as fcc_file:
        data = json.load(fcc_file)
        
        data_dict["list_{}".format(model_name)] = []

        for key in data.keys():
            
            data_dict["list_{}".format(model_name)].append(data[key])

In [3]:
df = pd.DataFrame.from_dict(data_dict, orient='index').T
df

,list_model1,list_model2,list_model3,list_model4,list_model5,list_model6,list_model7,list_model8,list_model9
0,038900014278,5900907000965,0693493034463,030271034077,5905643070845,5905643071644,3069900201327,5900783007621,055369901703
1,5604912425040,7290000207162,7290000207438,5201940010110,6118000240501,6111049002055,5900907000101,5900907001412,5201940010011
2,5705550300963,7290000208077,7290000208008,5604912415058,7290000208022,7290000208343,6111049000051,693493034487,5400140094207
3,693493034517,7290000208336,7290000208244,693493034456,7290000208114,7290000208367,693493034470,693493034494,5604912425057
4,7290000208213,7290000303598,7290000208329,7290000032665,7290000208145,7290000208428,7290000208091,7290000207063,5900783007614
5,7290000208930,7290002057123,7290000209715,7290000208220,7290000208800,7290001901243,7290000208251,7290000208176,5905643070883
6,7290000210360,7290002057734,7290000209913,7290000208480,7290000208855,7290001901250,7290000208404,7290000208268,7290000209906
7,7290000210469,7290002057864,7290002057017,7290000208534,7290000304663,7290001901670,7290000209388,7290000208312,7290000209951
8,7290000307237,7290002111085,7290002057093,7290000208893,7290000307015,7290002057437,7290000210896,7290000208350,7290000218502
9,7290002019787,7290002111672,7290002057161,7290000208947,7290000307046,7290002111214,7290000305882,7290000208459,7290001482186


In [4]:
import os
import matplotlib
import shutil

test_data_path = '/home/orelit/Projects -Sathsara/Planigo/data/Canned_food_testing'

folder_names = []
for folder in os.listdir(test_data_path):
    folder_names.append(folder)

In [5]:
feature_extractor = ViTFeatureExtractor.from_pretrained('google/vit-base-patch16-224-in21k')

In [6]:
def prediction(img_path,wine_model):
   im=Image.open(img_path)
   encoding = feature_extractor(images=im, return_tensors="pt")
   encoding.keys()
   pixel_values = encoding['pixel_values']
   outputs = wine_model(pixel_values)
   result = outputs.logits.softmax(1).argmax(1)
   tensor_result = outputs.logits.softmax(1)
   prob = torch.max(tensor_result)
   new_result = result.tolist() 
   for i in new_result:
     return(id2label[str(i)])

In [7]:
from pathlib import Path
from torchvision.datasets import ImageFolder
from transformers import ViTFeatureExtractor, ViTForImageClassification
from PIL import Image, UnidentifiedImageError
import torch


test_results = {}
result_counts = {}
model_list = []
folder_image_counts = []
folder_correct_counts = []
ground_truth_list = []
predicted_list = []

count = 0
correct = 0
for folder in folder_names:
    prediction_list = []
    print('folder name :',folder)
    for column in df.columns:
        if folder  in df[column].unique():
            model  = column
    print('model name :',model)
    
    model_name = model[5:]
    print(model_name)
    
    data_dir = Path("/home/orelit/Projects -Sathsara/Planigo/data/Canned_food_Splitted/{}".
                    format(model_name))
    print(data_dir)
    ds=ImageFolder(data_dir)
    
    label2id = {}
    id2label = {}
    for i, class_name in enumerate(ds.classes):
        label2id[class_name] = str(i)
        id2label[str(i)] = class_name
        
    model_path = '/home/orelit/Projects -Sathsara/Planigo/Models/VIT_CF/{}/{}'.format(model_name,model_name)
    wine_model = ViTForImageClassification.from_pretrained(
         model_path,
         num_labels=len(label2id),
         label2id=label2id,
         id2label=id2label)
    folder_correct_count = 0
    folder_image_count = 0
    for image in os.listdir(test_data_path+'/'+folder):
        print('image name : ',image)
        img = test_data_path+'/'+folder +'/'+image
        pred= prediction(img,wine_model)
        predicted_list.append(pred)
        print("predicted label : ",pred)
        test_label = image[image.rfind('/')+1:]
        real_class = test_label[0:13]
        ground_truth_list.append(real_class)
        print('real label : ',real_class)
        folder_image_count = folder_image_count +1
        if real_class == pred:
            correct = correct +1
            folder_correct_count = folder_correct_count +1
            print("Correctly classified")
            prediction_list.append('Correctly Classified')
        else:
            prediction_list.append(pred)
        count = count + 1
        print('..........................................')
        
    model_list.append(model[5:])
    folder_correct_counts.append(folder_correct_count)
    folder_image_counts.append(folder_image_count)
    
    test_results[folder] = prediction_list

acc = correct/count
print("Number of images : ",count)
print("Number of correctly classified images : ",correct)
print("Accuray : ",acc)
print(test_results)

folder name : 7290008175814
model name : list_model9
model9
/home/orelit/Projects -Sathsara/Planigo/data/Canned_food_Splitted/model9
image name :  7290008175814 (35).jpeg
predicted label :  7290008175814
real label :  7290008175814
Correctly classified
..........................................
image name :  7290008175814 (12).jpeg
predicted label :  7290008175814
real label :  7290008175814
Correctly classified
..........................................
image name :  7290008175814 (61).jpeg
predicted label :  7290008175814
real label :  7290008175814
Correctly classified
..........................................
image name :  7290008175814 (47).jpeg
predicted label :  7290008175814
real label :  7290008175814
Correctly classified
..........................................
image name :  7290008175814 (14).jpeg
predicted label :  7290008175814
real label :  7290008175814
Correctly classified
..........................................
image name :  7290008175814 (9).jpeg
predicted label

predicted label :  7290008175814
real label :  7290008175814
Correctly classified
..........................................
image name :  7290008175814 (53).jpeg
predicted label :  7290008175814
real label :  7290008175814
Correctly classified
..........................................
image name :  7290008175814 (7).jpeg
predicted label :  7290008175814
real label :  7290008175814
Correctly classified
..........................................
image name :  7290008175814 (62).jpeg
predicted label :  7290008175814
real label :  7290008175814
Correctly classified
..........................................
image name :  7290008175814 (30).jpeg
predicted label :  7290008175814
real label :  7290008175814
Correctly classified
..........................................
image name :  7290008175814 (11).jpeg
predicted label :  7290008175814
real label :  7290008175814
Correctly classified
..........................................
image name :  7290008175814 (45).jpeg
predicted label :  7290

predicted label :  7290003578177
real label :  7290003578177
Correctly classified
..........................................
image name :  7290003578177 (32).jpeg
predicted label :  7290003578177
real label :  7290003578177
Correctly classified
..........................................
image name :  7290003578177 (1).jpeg
predicted label :  7290003578177
real label :  7290003578177
Correctly classified
..........................................
image name :  7290003578177 (27).jpeg
predicted label :  7290003578177
real label :  7290003578177
Correctly classified
..........................................
image name :  7290003578177 (28).jpeg
predicted label :  7290003578177
real label :  7290003578177
Correctly classified
..........................................
image name :  7290003578177 (6).jpeg
predicted label :  7290003578177
real label :  7290003578177
Correctly classified
..........................................
image name :  7290003578177 (9).jpeg
predicted label :  729000

predicted label :  7290002929642
real label :  7290002929642
Correctly classified
..........................................
image name :  7290002929642 (12).jpeg
predicted label :  7290002929642
real label :  7290002929642
Correctly classified
..........................................
image name :  7290002929642 (22).jpeg
predicted label :  7290002929642
real label :  7290002929642
Correctly classified
..........................................
image name :  7290002929642 (24).jpeg
predicted label :  7290002929642
real label :  7290002929642
Correctly classified
..........................................
image name :  7290002929642 (8).jpeg
predicted label :  7290002929642
real label :  7290002929642
Correctly classified
..........................................
image name :  7290002929642 (16).jpeg
predicted label :  7290002929642
real label :  7290002929642
Correctly classified
..........................................
image name :  7290002929642 (11).jpeg
predicted label :  7290

predicted label :  7290003550753
real label :  7290003550753
Correctly classified
..........................................
image name :  7290003550753 (9).jpeg
predicted label :  7290003550753
real label :  7290003550753
Correctly classified
..........................................
image name :  7290003550753 (21).jpeg
predicted label :  7290003550753
real label :  7290003550753
Correctly classified
..........................................
image name :  7290003550753 (41).jpeg
predicted label :  7290003550753
real label :  7290003550753
Correctly classified
..........................................
image name :  7290003550753 (20).jpeg
predicted label :  7290003550753
real label :  7290003550753
Correctly classified
..........................................
image name :  7290003550753 (51).jpeg
predicted label :  7290003550753
real label :  7290003550753
Correctly classified
..........................................
image name :  7290003550753 (26).jpeg
predicted label :  7290

predicted label :  7290003578061
real label :  7290003578061
Correctly classified
..........................................
image name :  7290003578061 (22).jpeg
predicted label :  7290003578061
real label :  7290003578061
Correctly classified
..........................................
image name :  7290003578061 (2).jpeg
predicted label :  7290003578061
real label :  7290003578061
Correctly classified
..........................................
image name :  7290003578061 (1).jpeg
predicted label :  7290003578061
real label :  7290003578061
Correctly classified
..........................................
image name :  7290003578061 (29).jpeg
predicted label :  7290003578061
real label :  7290003578061
Correctly classified
..........................................
image name :  7290003578061 (45).jpeg
predicted label :  7290003578061
real label :  7290003578061
Correctly classified
..........................................
image name :  7290003578061 (3).jpeg
predicted label :  729000

predicted label :  7290010175857
real label :  7290010175857
Correctly classified
..........................................
image name :  7290010175857 (7).jpeg
predicted label :  7290010175857
real label :  7290010175857
Correctly classified
..........................................
image name :  7290010175857 (19).jpeg
predicted label :  7290010175857
real label :  7290010175857
Correctly classified
..........................................
image name :  7290010175857 (17).jpeg
predicted label :  7290010175857
real label :  7290010175857
Correctly classified
..........................................
image name :  7290010175857 (24).jpeg
predicted label :  7290010175857
real label :  7290010175857
Correctly classified
..........................................
image name :  7290010175857 (2).jpeg
predicted label :  7290010175857
real label :  7290010175857
Correctly classified
..........................................
image name :  7290010175857 (10).jpeg
predicted label :  72900

predicted label :  7290002057628
real label :  7290016370225
..........................................
image name :  7290016370225 (10).jpeg
predicted label :  7290016517538
real label :  7290016370225
..........................................
folder name : 7290006511157
model name : list_model4
model4
/home/orelit/Projects -Sathsara/Planigo/data/Canned_food_Splitted/model4
image name :  7290006511157 (9).jpeg
predicted label :  7290006511157
real label :  7290006511157
Correctly classified
..........................................
image name :  7290006511157 (7).jpeg
predicted label :  7290006511157
real label :  7290006511157
Correctly classified
..........................................
image name :  7290006511157 (5).jpeg
predicted label :  7290006511157
real label :  7290006511157
Correctly classified
..........................................
image name :  7290006511157 (8).jpeg
predicted label :  7290006511157
real label :  7290006511157
Correctly classified
................

image name :  7290004219079 (13).jpeg
predicted label :  7290004219079
real label :  7290004219079
Correctly classified
..........................................
image name :  7290004219079 (25).jpeg
predicted label :  7290004219079
real label :  7290004219079
Correctly classified
..........................................
image name :  7290004219079 (11).jpeg
predicted label :  7290004219079
real label :  7290004219079
Correctly classified
..........................................
image name :  7290004219079 (34).jpeg
predicted label :  7290004219079
real label :  7290004219079
Correctly classified
..........................................
image name :  7290004219079 (28).jpeg
predicted label :  7290004219079
real label :  7290004219079
Correctly classified
..........................................
image name :  7290004219079 (22).jpeg
predicted label :  7290004219079
real label :  7290004219079
Correctly classified
..........................................
image name :  72900042

In [8]:
result_counts = pd.DataFrame(list(zip(folder_names, model_list,folder_image_counts,folder_correct_counts)),
               columns =['folder_names', 'model_list','folder_image_count','folder_correct_count'])
result_counts

,folder_names,model_list,folder_image_count,folder_correct_count
0,7290008175814,model9,62,62
1,7290002157960,model3,12,12
2,7290006511751,model2,7,7
3,7290005287527,model2,10,0
4,7290003578177,model4,39,38
5,7290000450308,model5,2,2
6,7290002929642,model6,36,36
7,7290003550753,model9,51,51
8,7290016401264,model4,6,3
9,7290016401271,model1,9,0


In [9]:
incorrect_list = []
for i,row in enumerate(result_counts['folder_names']):
    incorrect = result_counts['folder_image_count'][i] - result_counts['folder_correct_count'][i]
    incorrect_list.append(incorrect )
    
incorrect_list

[0, 0, 0, 10, 1, 0, 0, 0, 3, 9, 0, 5, 0, 0, 10, 1, 0, 4, 0]

In [10]:
result_counts['incorrectly classified'] = incorrect_list
result_counts

,folder_names,model_list,folder_image_count,folder_correct_count,incorrectly classified
0,7290008175814,model9,62,62,0
1,7290002157960,model3,12,12,0
2,7290006511751,model2,7,7,0
3,7290005287527,model2,10,0,10
4,7290003578177,model4,39,38,1
5,7290000450308,model5,2,2,0
6,7290002929642,model6,36,36,0
7,7290003550753,model9,51,51,0
8,7290016401264,model4,6,3,3
9,7290016401271,model1,9,0,9


In [32]:
result_counts.to_csv('/home/orelit/Projects -Sathsara/Planigo/Test results/Canned Food/result_counts.csv')

In [11]:
results_df = pd.DataFrame.from_dict(test_results, orient='index').T

In [12]:
results_df.fillna("",inplace=True)
results_df

,7290008175814,7290002157960,7290006511751,7290005287527,7290003578177,7290000450308,7290002929642,7290003550753,7290016401264,7290016401271,7290003578061,7290005287534,7290010175857,7290002929284,7290016370225,7290006511157,7290002157953,7290010175840,7290004219079
0,Correctly Classified,Correctly Classified,Correctly Classified,7290013847836,Correctly Classified,Correctly Classified,Correctly Classified,Correctly Classified,Correctly Classified,7290017617046,Correctly Classified,Correctly Classified,Correctly Classified,Correctly Classified,7290016517538,Correctly Classified,Correctly Classified,Correctly Classified,Correctly Classified
1,Correctly Classified,Correctly Classified,Correctly Classified,7290013847836,Correctly Classified,Correctly Classified,Correctly Classified,Correctly Classified,Correctly Classified,7290017617046,Correctly Classified,7290113195196,Correctly Classified,Correctly Classified,7290016517538,Correctly Classified,Correctly Classified,Correctly Classified,Correctly Classified
2,Correctly Classified,Correctly Classified,Correctly Classified,7290013847836,Correctly Classified,,Correctly Classified,Correctly Classified,693493034456,693493034517,Correctly Classified,Correctly Classified,Correctly Classified,Correctly Classified,7290002057628,Correctly Classified,Correctly Classified,Correctly Classified,Correctly Classified
3,Correctly Classified,Correctly Classified,Correctly Classified,7290013847836,Correctly Classified,,Correctly Classified,Correctly Classified,7290019297796,693493034517,Correctly Classified,7290108507942,Correctly Classified,Correctly Classified,7290016401424,Correctly Classified,Correctly Classified,7290113192324,Correctly Classified
4,Correctly Classified,Correctly Classified,Correctly Classified,7290013847836,Correctly Classified,,Correctly Classified,Correctly Classified,Correctly Classified,7290017617046,Correctly Classified,Correctly Classified,Correctly Classified,Correctly Classified,7290002057628,Correctly Classified,Correctly Classified,7290113192324,Correctly Classified
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57,Correctly Classified,,,,,,,,,,,,,,,,,,
58,Correctly Classified,,,,,,,,,,,,,,,,,,
59,Correctly Classified,,,,,,,,,,,,,,,,,,
60,Correctly Classified,,,,,,,,,,,,,,,,,,


In [31]:
results_df.to_csv('/home/orelit/Projects -Sathsara/Planigo/Test results/Canned Food/results.csv')

In [6]:
def prediction(img_path,wine_model):
   im=Image.open(img_path)
   encoding = feature_extractor(images=im, return_tensors="pt")
   encoding.keys()
   pixel_values = encoding['pixel_values']
   outputs = wine_model(pixel_values)
   result = outputs.logits.softmax(1).argmax(1)
   tensor_result = outputs.logits.softmax(1)
   prob = torch.max(tensor_result)
   float_result = prob.type(torch.DoubleTensor)
   new_result = result.tolist() 
   for i in new_result:
     return((id2label[str(i)],float_result))

In [8]:
total= 0
correct = 0
for folder in folder_names:
    for image in os.listdir(test_data_path+'/'+folder):
        total = total + 1
        model_results = {}
        for model in model_names:
            data_dir = Path("/home/orelit/Projects -Sathsara/Planigo/data/Canned_food_Splitted/{}".
                    format(model))
#             print(data_dir)
            ds=ImageFolder(data_dir)
        
            label2id = {}
            id2label = {}
            for i, class_name in enumerate(ds.classes):
                label2id[class_name] = str(i)
                id2label[str(i)] = class_name
            
            model_path = '/home/orelit/Projects -Sathsara/Planigo/Models/VIT_CF/{}/{}'.format(model,model)
            wine_model = ViTForImageClassification.from_pretrained(
                   model_path,
                   num_labels=len(label2id),
                   label2id=label2id,
                   id2label=id2label)
            img = test_data_path+'/'+folder +'/'+image
            pred,confidence= prediction(img,wine_model)
#             print("predicted label : ",pred) 
#             print("confidence : ",confidence) 
            model_results[pred] = confidence
        
#         print(model_results)
        probs = list(model_results.values())
#         print("PROBABILITIES : ",probs)
        maximum = probs[0]
        index = 0
        for i in range(1,len( probs )):
            if  probs [i] > maximum:
                maximum = probs[i]
                index = i
        labels = list(model_results.keys())
        predicted_label = labels[index]
#         print("LABELS",predicted_label)
        print(image)
        real_class = image[0:13]
        if real_class==predicted_label:
            print("Correctly Predicted")
            correct = correct + 1
        print('...........................................................................')
        
accuracy = correct/total

7290008175814 (35).jpeg
Correctlyt Predicted
...........................................................................
7290008175814 (12).jpeg
Correctlyt Predicted
...........................................................................
7290008175814 (61).jpeg
Correctlyt Predicted
...........................................................................
7290008175814 (47).jpeg
Correctlyt Predicted
...........................................................................
7290008175814 (14).jpeg
Correctlyt Predicted
...........................................................................
7290008175814 (9).jpeg
Correctlyt Predicted
...........................................................................
7290008175814 (1).jpeg
Correctlyt Predicted
...........................................................................
7290008175814 (41).jpeg
Correctlyt Predicted
...........................................................................
7290008175814 (56).jpeg
Correctlyt

7290002157960 (4).jpeg
Correctlyt Predicted
...........................................................................
7290002157960 (9).jpeg
Correctlyt Predicted
...........................................................................
7290002157960 (7).jpeg
Correctlyt Predicted
...........................................................................
7290002157960 (12).jpeg
Correctlyt Predicted
...........................................................................
7290002157960 (5).jpeg
Correctlyt Predicted
...........................................................................
7290002157960 (11).jpeg
Correctlyt Predicted
...........................................................................
7290006511751 (4).jpeg
Correctlyt Predicted
...........................................................................
7290006511751 (2).jpeg
Correctlyt Predicted
...........................................................................
7290006511751 (1).jpeg
Correctlyt Pred

7290002929642 (3).jpeg
Correctlyt Predicted
...........................................................................
7290002929642 (6).jpeg
Correctlyt Predicted
...........................................................................
7290002929642 (7).jpeg
Correctlyt Predicted
...........................................................................
7290002929642 (9).jpeg
Correctlyt Predicted
...........................................................................
7290002929642 (28).jpeg
Correctlyt Predicted
...........................................................................
7290002929642 (25).jpeg
Correctlyt Predicted
...........................................................................
7290002929642 (27).jpeg
Correctlyt Predicted
...........................................................................
7290002929642 (10).jpeg
...........................................................................
7290002929642 (1).jpeg
Correctlyt Predicted
.............

7290003550753 (20).jpeg
Correctlyt Predicted
...........................................................................
7290003550753 (51).jpeg
Correctlyt Predicted
...........................................................................
7290003550753 (26).jpeg
Correctlyt Predicted
...........................................................................
7290003550753 (47).jpeg
Correctlyt Predicted
...........................................................................
7290003550753 (35).jpeg
Correctlyt Predicted
...........................................................................
7290003550753 (46).jpeg
Correctlyt Predicted
...........................................................................
7290003550753 (24).jpeg
Correctlyt Predicted
...........................................................................
7290003550753 (36).jpeg
Correctlyt Predicted
...........................................................................
7290003550753 (15).jpeg
Correctl

7290003578061 (42).jpeg
Correctlyt Predicted
...........................................................................
7290003578061 (26).jpeg
Correctlyt Predicted
...........................................................................
7290005287534 (1).jpeg
...........................................................................
7290005287534 (13).jpeg
...........................................................................
7290005287534 (10).jpeg
...........................................................................
7290005287534 (14).jpeg
...........................................................................
7290005287534 (4).jpeg
...........................................................................
7290005287534 (9).jpeg
...........................................................................
7290005287534 (2).jpeg
...........................................................................
7290005287534 (3).jpeg
.......................................

7290016370225 (3).jpeg
...........................................................................
7290016370225 (1).jpeg
...........................................................................
7290016370225 (5).jpeg
...........................................................................
7290016370225 (7).jpeg
...........................................................................
7290016370225 (10).jpeg
...........................................................................
7290006511157 (9).jpeg
...........................................................................
7290006511157 (7).jpeg
...........................................................................
7290006511157 (5).jpeg
...........................................................................
7290006511157 (8).jpeg
...........................................................................
7290006511157 (12).jpeg
...........................................................................
72900065

7290004219079 (30).jpeg
Correctlyt Predicted
...........................................................................
7290004219079 (18).jpeg
Correctlyt Predicted
...........................................................................
7290004219079 (5).jpeg
Correctlyt Predicted
...........................................................................
7290004219079 (12).jpeg
Correctlyt Predicted
...........................................................................
7290004219079 (15).jpeg
Correctlyt Predicted
...........................................................................
7290004219079 (17).jpeg
Correctlyt Predicted
...........................................................................
7290004219079 (3).jpeg
Correctlyt Predicted
...........................................................................
7290004219079 (21).jpeg
Correctlyt Predicted
...........................................................................
7290004219079 (7).jpeg
Correctlyt 

In [9]:
print(accuracy )

0.748868778280543
